Pre reqs

In [ ]:
!pip install inference-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.4 MB/s eta 0:00:00


Importing Libraries

In [ ]:
from inference_sdk import InferenceHTTPClient
import numpy as np
from PIL import Image
import requests
import io

Functions Definitions

In [ ]:
def api_predict(image):
    CLIENT = InferenceHTTPClient(
    api_url="https://classify.roboflow.com",
    api_key="rF6T4VHoa0dFRGvtKQS0")

    response = CLIENT.infer(image, model_id="wiki-art/2")

    return (response['predictions'][0]['class_id'])

def targeted_attack(image_path, epsilon, alpha, num_iter,actual_class):
    # Load the image
    image = Image.open(image_path).convert('RGB')
    image_np = np.array(image, dtype=np.float32) / 255.0

    for _ in range(num_iter):
        # Perturb the image slightly
        perturbation = np.random.uniform(-alpha, alpha, image_np.shape)
        image_np = np.clip(image_np + perturbation, 0, 1)

        # Convert back to PIL image
        perturbed_image = Image.fromarray((image_np * 255).astype(np.uint8))

        # Check the prediction
        predicted_class = api_predict(perturbed_image)

        # Break the loop if we achieve the target class
        if predicted_class != actual_class:
            print(f"Target class achieved after {_+1} iterations")
            break

    # Save the final adversarial image
    perturbed_image.save('adversarial_image.png')
    return perturbed_image

def add_noise(image_path,epsilon,alpha): #0.1 & 0.01
  actual_class = api_predict(image_path)
  num_iter = 1000

  adversarial_image = targeted_attack(image_path, epsilon, alpha, num_iter,actual_class)
  return adversarial_image

Noise Addition


In [ ]:
image_path=input("Enter the image path: ")
print("Enter parameters for noise addition: ")
epsilon = float(input("Epsilon: "))
alpha = float(input("Alpha: "))

noisy=add_noise(image_path,epsilon,alpha)

Enter the image path: /content/naive.jpeg
Enter parameters for noise addition: 
Epsilon: 0.1
Alpha: 0.01
Target class achieved after 141 iterations
